# LiqPay API Notebook

## Setup

Load environment variables and code autoreload plugin.

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

Imports

In [ ]:
import json

from os import getenv
from uuid import uuid4
from pprint import pprint
from datetime import date, datetime, timedelta, UTC

from liqpy.testing import TestCard, gen_card_cvv, gen_card_expire

Initialize client with public and private keys

In [ ]:
from liqpy.client import Client

client = Client()
client

## Examples

Encode request and decode it as callback

In [ ]:
params, signature = client.encode(
    action="pay",
    amount=1,
    order_id=uuid4(),
    description="Test Encoding",
    currency="USD",
)

sep, end = "\n", "\n\n"
print("data:", params, sep=sep, end=end)
print("signature:", signature, sep=sep, end=end)

client.callback(params, signature)

Pay request with test card

In [ ]:
card_exp_month, card_exp_year = gen_card_expire(valid=True)

order_id = uuid4()
client.pay(
    order_id=order_id,
    amount=1,
    currency="USD",
    description="Test Payment",
    card=TestCard.successful(),
    card_exp_month=card_exp_month,
    card_exp_year=card_exp_year,
    card_cvv=gen_card_cvv(),
)

Refund the payment

In [ ]:
client.refund(order_id=order_id, amount=1)

Pay request with fail

In [ ]:
from liqpy.exceptions import LiqPayException

try:
    card_exp_month, card_exp_year = gen_card_expire(valid=True)

    order_id = str(uuid4())
    client.pay(
        order_id=order_id,
        amount=1,
        currency="USD",
        description="Test Payment",
        card=TestCard.failure("9859"),
        card_exp_month=card_exp_month,
        card_exp_year=card_exp_year,
        card_cvv=gen_card_cvv(),
    )
except LiqPayException as e:
    print(e.response)
    pprint(e.details)
    raise e

Get a payment status

In [ ]:
client.status(order_id)

add info to payment

In [ ]:
client.data(order_id, "Lorem Ipsum")

Create checkout link

In [ ]:
client.checkout(
    "pay",
    amount=1,
    order_id=str(uuid4()),
    description="Test Checkout",
    currency="USD",
)

Test checkout callback

In [ ]:
from liqpy.server import LiqpayServer
from webbrowser import open_new_tab

order_id = uuid4()
checkout_url = client.checkout(
    "pay",
    amount=1,
    order_id=uuid4(),
    description="Test Checkout",
    currency="USD",
    server_url=getenv("SERVER_URL", ""),
    expired_date=(datetime.now(UTC) + timedelta(seconds=30)),
)

open_new_tab(checkout_url)
print("checkout link", checkout_url)

with LiqpayServer(client=client, timeout=60) as server:
    server.handle_request()

Get payments report

In [ ]:
params[3100:]

In [ ]:
date_to = datetime.now(UTC)
date_from = date_to - timedelta(days=1)
print("from:", date_from)
print("to:", date_to)

params = client.reports(date_from=date_from, date_to=date_to, format="json")
params = json.loads(params)
params

In [ ]:
print(client.reports(date_from=date_from, date_to=date_to, format="csv"))

In [ ]:
print(client.reports(date_from=date_from, date_to=date_to, format="xml"))

Create subscription

In [ ]:
card_exp_month, card_exp_year = gen_card_expire(valid=True)

order_id = uuid4()
client.subscribe(
    amount=1,
    order_id=order_id,
    description="Test Subscribe",
    currency="USD",
    card=TestCard.successful(),
    card_exp_month=card_exp_month,
    card_exp_year=card_exp_year,
    card_cvv=gen_card_cvv(),
    phone="+380661234567",
    subscribe_periodicity="month",
)

Unsubscribe

In [ ]:
client.unsubscribe(order_id=order_id)